In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mohitsingh1804/plantvillage

100% 817M/818M [00:35<00:00, 24.3MB/s]
100% 818M/818M [00:35<00:00, 23.9MB/s]


In [ ]:
!unzip plantvillage.zip

Streaming output truncated to the last 5000 lines.
  inflating: PlantVillage/val/Soybean___healthy/52c7d07b-561b-4380-801a-3824e65a5135___RS_HL 3797.JPG  
  inflating: PlantVillage/val/Soybean___healthy/53168d1c-ac90-4039-9eff-7479475a6b74___RS_HL 6355.JPG  
  inflating: PlantVillage/val/Soybean___healthy/5344b09f-bbbe-4a2c-9345-d93dd4e69eb4___RS_HL 4693.JPG  
  inflating: PlantVillage/val/Soybean___healthy/5361ba4a-351e-466c-99c5-8845f35c00df___RS_HL 2890.JPG  
  inflating: PlantVillage/val/Soybean___healthy/537e5409-d824-4859-967f-3c502f13deec___RS_HL 5474.JPG  
  inflating: PlantVillage/val/Soybean___healthy/538969da-545b-4bce-bc1d-0932898dc5e6___RS_HL 5230.JPG  
  inflating: PlantVillage/val/Soybean___healthy/538ccfe2-1509-40d7-beae-ae7cf84e1e5d___RS_HL 3717.JPG  
  inflating: PlantVillage/val/Soybean___healthy/5397bf6a-aeeb-4f1c-b554-bdb604bb23c3___RS_HL 6508.JPG  
  inflating: PlantVillage/val/Soybean___healthy/53dc0466-621c-43e1-9c5e-e340f3d50171___RS_HL 6381.JPG  
  inflating: 

In [ ]:
!pip install efficientnet
!pip install tensorflow-addons==0.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense,Conv2D,MaxPool2D,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.metrics import AUC, CategoricalAccuracy, Precision, Recall
import tensorflow as tf
import cv2

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
epochs = 30

In [ ]:
def remove(path):
    """ param <path> could either be relative or absolute. """
    if os.path.isfile(path) or os.path.islink(path):
        os.remove(path)  # remove the file
    elif os.path.isdir(path):
        shutil.rmtree(path)  # remove dir and all contains
    else:
        raise ValueError("file {} is not a file or dir.".format(path))

In [ ]:
import shutil

from os import listdir
from os.path import join, isfile
files = listdir('PlantVillage/train/')


for f in files:
  if f[0] != 'T':
    print(f)
    remove('PlantVillage/train/'+f)

files = listdir('PlantVillage/train/')
files = [f.lower() for f in files]


files = listdir('PlantVillage/val/')


for f in files:
  if f[0] != 'T':
    print(f)
    remove('PlantVillage/val/'+f)

files = listdir('PlantVillage/val/')
files = [f.lower() for f in files]

sorted(files)

Cherry_(including_sour)___healthy
Apple___Black_rot
Grape___Black_rot
Pepper,_bell___Bacterial_spot
Potato___Early_blight
Soybean___healthy
Raspberry___healthy
Pepper,_bell___healthy
Corn_(maize)___Northern_Leaf_Blight
Strawberry___Leaf_scorch
Apple___Apple_scab
Peach___Bacterial_spot
Grape___healthy
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Potato___Late_blight
Squash___Powdery_mildew
Peach___healthy
Corn_(maize)___healthy
Cherry_(including_sour)___Powdery_mildew
Orange___Haunglongbing_(Citrus_greening)
Strawberry___healthy
Grape___Esca_(Black_Measles)
Apple___healthy
Blueberry___healthy
Apple___Cedar_apple_rust
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Potato___healthy
Corn_(maize)___Common_rust_
Cherry_(including_sour)___healthy
Apple___Black_rot
Grape___Black_rot
Pepper,_bell___Bacterial_spot
Potato___Early_blight
Soybean___healthy
Raspberry___healthy
Pepper,_bell___healthy
Corn_(maize)___Northern_Leaf_Blight
Strawberry___Leaf_scorch
Apple___Apple_scab
Peach___Bacterial_s

['tomato___bacterial_spot',
 'tomato___early_blight',
 'tomato___healthy',
 'tomato___late_blight',
 'tomato___leaf_mold',
 'tomato___septoria_leaf_spot',
 'tomato___spider_mites two-spotted_spider_mite',
 'tomato___target_spot',
 'tomato___tomato_mosaic_virus',
 'tomato___tomato_yellow_leaf_curl_virus']

In [ ]:
label =files
NUM_CLASS=len(label)

In [ ]:
train_gen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input
#     preprocessing_function = efn.preprocess_input
)

# Only rescaling the val set
val_gen = ImageDataGenerator(
    rescale=1/255.0,
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input
#     preprocessing_function = efn.preprocess_input
)

In [ ]:
train_generator = train_gen.flow_from_directory(
    directory="PlantVillage/train/",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 14529 images belonging to 10 classes.


In [ ]:
valid_generator = val_gen.flow_from_directory(
    directory='PlantVillage/val/',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
)

Found 3631 images belonging to 10 classes.


In [ ]:
def see_examples(datagen, figsize=(16,16),nrows=4, ncols=4):
    fig, ax = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols)
    batch_idx = np.random.randint(low=0, high=len(datagen), size=(nrows,ncols))
    for i in range(nrows):
        for j in range(ncols):
            imgs, labels = datagen[batch_idx[i,j]]
            sample_idx = np.random.randint(low=0, high=len(imgs))
            img = imgs[sample_idx]
            label = labels[sample_idx]
            label = class_dict[np.argmax(label)]
            ax[i,j].axis('off')
            ax[i,j].imshow(img)
            ax[i,j].set_title(label)
    plt.show()

In [ ]:
conv_base = efn.EfficientNetB0(weights='imagenet',
                  include_top=False,
                  input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))
conv_base.trainable=False

16804768/16804768 [==============================] - 1s 0us/step


In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block6a_expand_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model = Sequential([
    conv_base,
    GlobalAveragePooling2D(),

    Dense(units=512,activation='relu'),
    Dropout(0.2),
    Dense(units=64, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dense(units=NUM_CLASS, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functiona  (None, 7, 7, 1280)        4049564   
 l)                                                              
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [ ]:
callbacks = [
    EarlyStopping(min_delta=1e-5, patience=15),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=0.001)
]

metrics = [
    CategoricalAccuracy(),
    Precision(),
    Recall(),
#     AUC()
]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
history = model.fit(train_generator,epochs=epochs, batch_size=BATCH_SIZE,
                  validation_data=valid_generator,callbacks= callbacks
                             )

Epoch 1/30
228/228 [==============================] - ETA: 0s - loss: 0.4214 - categorical_accuracy: 0.8654 - precision: 0.9276 - recall: 0.8186

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


228/228 [==============================] - 227s 879ms/step - loss: 0.4214 - categorical_accuracy: 0.8654 - precision: 0.9276 - recall: 0.8186 - val_loss: 0.1522 - val_categorical_accuracy: 0.9565 - val_precision: 0.9593 - val_recall: 0.9551 - lr: 0.0010
Epoch 2/30
228/228 [==============================] - 198s 867ms/step - loss: 0.1430 - categorical_accuracy: 0.9571 - precision: 0.9638 - recall: 0.9518 - val_loss: 0.0974 - val_categorical_accuracy: 0.9714 - val_precision: 0.9740 - val_recall: 0.9697 - lr: 0.0010
Epoch 3/30
228/228 [==============================] - 205s 898ms/step - loss: 0.1150 - categorical_accuracy: 0.9652 - precision: 0.9702 - recall: 0.9619 - val_loss: 0.1241 - val_categorical_accuracy: 0.9617 - val_precision: 0.9648 - val_recall: 0.9584 - lr: 0.0010
Epoch 4/30
228/228 [==============================] - 199s 871ms/step - loss: 0.1011 - categorical_accuracy: 0.9691 - precision: 0.9737 - recall: 0.9665 - val_loss: 0.1419 - val_categorical_accuracy: 0.9650 - val_pre

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

,loss,categorical_accuracy,precision,recall,val_loss,val_categorical_accuracy,val_precision,val_recall,lr
0,0.421351,0.865442,0.927619,0.818570,0.152193,0.956486,0.959336,0.955109,0.001
1,0.143012,0.957120,0.963758,0.951752,0.097364,0.971358,0.973997,0.969705,0.001
2,0.114967,0.965242,0.970217,0.961869,0.124105,0.961719,0.964791,0.958414,0.001
3,0.101085,0.969096,0.973651,0.966481,0.141938,0.965023,0.967358,0.963096,0.001
4,0.074237,0.978182,0.981521,0.976117,0.073550,0.979895,0.981241,0.979620,0.001


In [ ]:
from google.colab import drive
drive.mount('/content/drive')